In [1]:
import requests

In [11]:
#we're going to use the Requests python module, which is a simple library that supports HTTP. 
# https://docs.python-requests.org/en/latest/
# the form of the basic call is 

# `requests.get(URL)`

# let's use a simple webpage call to see how it works

requests.get("http://www.example.com/")

<Response [200]>

In [27]:
# without storing the response object in a variable, it just outputs the first attribute listed, 
# which is a http response status code.  in this case, 200 means the request succeeded
# let's run that get request again, but this time store it in a variable

response = requests.get("http://www.example.com/")

In [28]:
# now we can look at the methods available to us using dot notation

# `response. /tab`

# and, let's just select `text` to see what was stored.
response.text

'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    <

In [29]:
# we can look at the headers to discover a little more about the object
response.headers

{'Content-Encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Age': '370752', 'Cache-Control': 'max-age=604800', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Sat, 25 Dec 2021 17:00:21 GMT', 'Etag': '"3147526947"', 'Expires': 'Sat, 01 Jan 2022 17:00:21 GMT', 'Last-Modified': 'Thu, 17 Oct 2019 07:18:26 GMT', 'Server': 'ECS (sab/5798)', 'Vary': 'Accept-Encoding', 'X-Cache': 'HIT', 'Content-Length': '648'}

In [30]:
# The data in the object is structured, but can end up being quite large, so let's use a dataframe 
# to help view the content

import pandas as pd

ModuleNotFoundError: No module named 'pandas'

The Zenodo REST API supports
- Deposit — upload and publishing of research outputs (identical to functionality available in the user interface).
- Records — search published records.
- Files — download/upload of files.

Zenodo's [quickstart guide](https://developers.zenodo.org/#quickstart-upload), which is the first thing on the page when looking at their documentation, describes how to upload, which requires a key.  We want to download, so we need to [jump to that section](https://developers.zenodo.org/#records) to find the URL structure we need to make an API records call.  Actually, the reference is buried a little in the documentation.  This is a good example of why you should familiarize yourself with an API's documentation prior to jumping right in.

The structure is

`GET /api/records/`

so our requests query will be 

In [22]:
response = requests.get("https://zenodo.org/api/records")

In [23]:
# we know from the developer documentation that the call returns a json object, 
# so to see the contents of the object we use
response.json()

{'aggregations': {'access_right': {'buckets': [{'doc_count': 2182865,
     'key': 'open'},
    {'doc_count': 47453, 'key': 'closed'},
    {'doc_count': 6591, 'key': 'restricted'},
    {'doc_count': 1400, 'key': 'embargoed'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0},
  'file_type': {'buckets': [{'doc_count': 995548, 'key': 'pdf'},
    {'doc_count': 371754, 'key': 'jpg'},
    {'doc_count': 362714, 'key': 'png'},
    {'doc_count': 231468, 'key': 'html'},
    {'doc_count': 110390, 'key': 'zip'},
    {'doc_count': 26392, 'key': 'xlsx'},
    {'doc_count': 23038, 'key': 'txt'},
    {'doc_count': 19247, 'key': 'docx'},
    {'doc_count': 18875, 'key': 'csv'},
    {'doc_count': 17926, 'key': 'xml'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 149102},
  'keywords': {'buckets': [{'doc_count': 906874, 'key': 'Taxonomy'},
    {'doc_count': 905692, 'key': 'Biodiversity'},
    {'doc_count': 566792, 'key': 'Animalia'},
    {'doc_count': 437668, 'key': 'Art

In [25]:
response.headers

{'Server': 'nginx', 'Date': 'Sat, 25 Dec 2021 16:59:17 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Vary': 'Accept-Encoding', 'Link': '<https://zenodo.org/api/records/?sort=mostrecent&page=1&size=10>; rel="self", <https://zenodo.org/api/records/?sort=mostrecent&page=2&size=10>; rel="next"', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '59', 'X-RateLimit-Reset': '1640451616', 'Retry-After': '58', 'X-Frame-Options': 'sameorigin', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Strict-Transport-Security': 'max-age=0', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Type, ETag, Link, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset', 'X-Request-ID': '0d71c5825f3967835c96182bb8b6053a', 'Content-Encoding': 'gzip'}